# Data Visualization Milestone 1 EDA Notebook

In [1]:
import pandas as pd
import numpy as np
import json
import gzip as gunzip
from matplotlib import pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 5000)
pd.set_option('display.max_rows', 5000)
from datetime import datetime
import pytz
import ast 
print(datetime.today().strftime("%d-%b-%Y %a"), "Notebook")
tz_Suisse = pytz.timezone('Europe/Zurich')
current_time= datetime.now(tz_Suisse)
print("Lausanne Time : ", current_time.strftime("%H:%M:%S"))

22-Mar-2024 Fri Notebook
Lausanne Time :  20:40:02


In [2]:
michelin_df = pd.read_csv(
    "/Users/ilkergul/Documents/EPFL/MA4/Data_visualization/com-480-project-datatouille/data/michelin_my_maps.csv")

michelin_df.head()

,Name,Address,Location,Price,Cuisine,Longitude,Latitude,PhoneNumber,Url,WebsiteUrl,Award,FacilitiesAndServices,Description
0,L'OSIER,"7-5-5 Ginza, Chuo-ku, Tokyo, 104-0061, Japan","Tokyo, Japan",¥¥¥¥,"French, Contemporary",139.761203,35.669667,8.133572e+10,https://guide.michelin.com/en/tokyo-region/tok...,https://losier.shiseido.co.jp/,3 Stars,"Air conditioning,Car park,Interesting wine list",In 2023 L’OSIER celebrated the 50th anniversar...
1,Azabu Kadowaki,"2-7-2 Azabujuban, Minato-ku, Tokyo, 106-0045, ...","Tokyo, Japan",¥¥¥¥,Japanese,139.733551,35.655993,8.135772e+10,https://guide.michelin.com/en/tokyo-region/tok...,NaN,3 Stars,"Air conditioning,Counter dining",Toshiya Kadowaki’s aim is to serve up “unforge...
2,Kohaku,"3-4 Kagurazaka, Shinjuku-ku, Tokyo, 162-0825, ...","Tokyo, Japan",¥¥¥¥,Japanese,139.741008,35.700701,8.150314e+11,https://guide.michelin.com/en/tokyo-region/tok...,http://kagurazaka-kohaku.jp/,3 Stars,"Air conditioning,Counter dining,Interesting wi...",Koji Koizumi creates a cuisine all his own as ...
3,Harutaka,"6F, 8-3-1 Ginza, Chuo-ku, Tokyo, 104-0061, Japan","Tokyo, Japan",¥¥¥¥,Sushi,139.759203,35.669269,8.133573e+10,https://guide.michelin.com/en/tokyo-region/tok...,NaN,3 Stars,"Air conditioning,Counter dining",Sometimes our connections intervene to change ...
4,Kanda,"1F, Toranomon Hills Residential Tower, 1-1-1 A...","Tokyo, Japan",¥¥¥¥,Japanese,139.748000,35.665800,8.136459e+10,https://guide.michelin.com/en/tokyo-region/tok...,http://www.nihonryori-kanda.com/,3 Stars,"Air conditioning,Counter dining,Interesting wi...",Hiroyuki Kanda understands the essence of Japa...


In [3]:
print(michelin_df.shape)

(6794, 13)


In [4]:
michelin_df.isna().sum()

Name                        0
Address                     0
Location                    0
Price                       2
Cuisine                     0
Longitude                   0
Latitude                    0
PhoneNumber               159
Url                         0
WebsiteUrl               1242
Award                       0
FacilitiesAndServices     234
Description                 0
dtype: int64

In [5]:
michelin_df.Award.value_counts()

Bib Gourmand    3333
1 Star          2844
2 Stars          474
3 Stars          143
Name: Award, dtype: int64

In [6]:
michelin_df.Location.value_counts()[:20]

Tokyo, Japan                      309
Kyoto, Japan                      193
Osaka, Japan                      189
New York, USA                     156
Paris, France                     156
Singapore                         132
London, United Kingdom            106
Hong Kong                          89
Seoul, South Korea                 89
Bangkok, Thailand                  87
Taipei, Taiwan                     80
Shanghai, China Mainland           77
Chicago, USA                       67
Guangzhou, China Mainland          61
Washington, USA                    54
Beijing, China Mainland            53
Hong Kong, Hong Kong SAR China     50
Madrid, Spain                      43
San Francisco, USA                 42
Nara, Japan                        42
Name: Location, dtype: int64

In [7]:
# Exact match replacement
michelin_df.loc[michelin_df['Location'] == 'Hong Kong', 'Location'] = 'Hong Kong, Hong Kong SAR China'

# To ensure that any variation of 'Hong Kong' including those with trailing spaces are also replaced
# You can use .str.replace() for a pattern match and replace
michelin_df['Location'] = michelin_df['Location'].str.replace(r'^Hong Kong$', 'Hong Kong, Hong Kong SAR China', regex=True)

# Filter rows where the Location includes 'Hong Kong'
hong_kong_rows = michelin_df[michelin_df.Location.str.contains("Hong Kong")]

# Display the filtered rows
hong_kong_rows.Award.value_counts()

Bib Gourmand    64
1 Star          56
2 Stars         12
3 Stars          7
Name: Award, dtype: int64

In [8]:
# Split the Location by comma and select the last part for the Country
michelin_df['Country'] = michelin_df['Location'].str.split(', ').str[-1]
michelin_df['Country'] = michelin_df['Country'].replace('Abu Dhabi', 'United Arab Emirates')
michelin_df['Country'] = michelin_df['Country'].replace('Dubai', 'United Arab Emirates')
michelin_df['Country'] = michelin_df['Country'].replace('Macau', 'Macau SAR China')
michelin_df['Country'] = michelin_df['Country'].replace('Czechia', 'Czech Republic')

# Extended mapping of country names to their flag emojis
flag_emojis = {
    'France': '🇫🇷',
    'Japan': '🇯🇵',
    'Italy': '🇮🇹',
    'USA': '🇺🇸',
    'Germany': '🇩🇪',
    'Spain': '🇪🇸',
    'United Kingdom': '🇬🇧',
    'Belgium': '🇧🇪',
    'Switzerland': '🇨🇭',
    'China Mainland': '🇨🇳',
    'Thailand': '🇹🇭',
    'Netherlands': '🇳🇱',
    'Taiwan': '🇹🇼',
    'Hong Kong SAR China': '🇭🇰',
    'Singapore': '🇸🇬',
    'South Korea': '🇰🇷',
    'Portugal': '🇵🇹',
    'Canada': '🇨🇦',
    'Malaysia': '🇲🇾',
    'Brazil': '🇧🇷',
    'Denmark': '🇩🇰',
    'United Arab Emirates': '🇦🇪',
    'Türkiye': '🇹🇷',
    'Vietnam': '🇻🇳',
    'Ireland': '🇮🇪',
    'Sweden': '🇸🇪',
    'Austria': '🇦🇹',
    'Croatia': '🇭🇷',
    'Macau SAR China': '🇲🇴',
    'Norway': '🇳🇴',
    'Greece': '🇬🇷',
    'Slovenia': '🇸🇮',
    'Luxembourg': '🇱🇺',
    'Hungary': '🇭🇺',
    'Argentina': '🇦🇷',
    'Poland': '🇵🇱',
    'Malta': '🇲🇹',
    'Finland': '🇫🇮',
    'Estonia': '🇪🇪',
    'Latvia': '🇱🇻',
    'Iceland': '🇮🇸',
    'Czech Republic': '🇨🇿',  # Note: Czechia and Czech Republic are the same
    'Serbia': '🇷🇸',
    'Andorra': '🇦🇩',
}

# Get the value counts of the countries and convert it to a DataFrame for Plotly
country_counts_df = michelin_df['Country'].value_counts().reset_index()
country_counts_df.columns = ['Country', 'Restaurant Count']

# Ensure the DataFrame is sorted by 'Restaurant Count' for better visual appearance
country_counts_df = country_counts_df.sort_values('Restaurant Count', ascending=False)
# Assume 'country_counts_df' has 'Country' column from your dataset
country_counts_df['Country With Flag'] = country_counts_df['Country'].apply(lambda x: f"{flag_emojis.get(x, '')} {x}")

In [9]:
import plotly.express as px

# Define a custom "Mint" color scale
michelin_inspired_color_scale = [
    [0.25, "#ffcccc"],  # Soluk kırmızı
    [0.5, "#ff9999"],  # Orta kırmızı
    [0.75, "#ff6666"],  # Daha koyu kırmızı
    [1, "#bf0000"]  # Michelin bordosu
]

base_height = 300
height_per_country = 20
fig_height = base_height + height_per_country * len(country_counts_df)

# Create a bar plot with improved layout for y-axis labels visibility
fig = px.bar(country_counts_df, 
             y='Country', 
             x='Restaurant Count', 
             orientation='h',
             title='Michelin Guide Listings by Country',
             template='simple_white',
             color='Restaurant Count',
             color_continuous_scale=michelin_inspired_color_scale,
             height=fig_height)  # Adjusted height based on the number of countries

# Customize hover data for additional interactivity and information richness
fig.update_traces(
    customdata=country_counts_df['Country'].apply(lambda x: flag_emojis.get(x, '')),
    hovertemplate="<b>Country:</b> %{y} %{customdata}<br><b>Restaurant Count:</b> %{x}<extra></extra>",
    marker=dict(line=dict(color='#EAEAEA', width=1))
)

# Improve layout for clarity and readability
fig.update_layout(
    xaxis_title='Number of Listings',
    yaxis_title='Country',
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Arial, sans-serif"
    ),
    title_font=dict(size=25, color='#333', family="Arial, sans-serif"),
    font=dict(family="Arial, sans-serif", color='#333'),
    paper_bgcolor='rgba(255,255,255,1)',
    plot_bgcolor='rgba(255,255,255,1)',
    margin=dict(t=100, l=150),  # Adjusted left margin for longer country names
    yaxis=dict(autorange="reversed", tickmode='linear')  # Ensure all country names are shown
)

# Optionally, adjust font size for the axis labels if needed
fig.update_yaxes(tickfont=dict(size=10))

# Show the figure
fig.show()


In [10]:
continent_mapping = {
    'France': 'Europe',
    'Japan': 'Asia',
    'Italy': 'Europe',
    'USA': 'North America',
    'Germany': 'Europe',
    'Spain': 'Europe',
    'United Kingdom': 'Europe',
    'Belgium': 'Europe',
    'Switzerland': 'Europe',
    'China Mainland': 'Asia',
    'Thailand': 'Asia',
    'Netherlands': 'Europe',
    'Taiwan': 'Asia',
    'Hong Kong SAR China': 'Asia',
    'Singapore': 'Asia',
    'South Korea': 'Asia',
    'Portugal': 'Europe',
    'Canada': 'North America',
    'Malaysia': 'Asia',
    'Brazil': 'South America',
    'Denmark': 'Europe',
    'United Arab Emirates': 'Asia',
    'Türkiye': 'Europe',
    'Vietnam': 'Asia',
    'Ireland': 'Europe',
    'Sweden': 'Europe',
    'Austria': 'Europe',
    'Croatia': 'Europe',
    'Macau SAR China': 'Asia',
    'Norway': 'Europe',
    'Greece': 'Europe',
    'Slovenia': 'Europe',
    'Luxembourg': 'Europe',
    'Hungary': 'Europe',
    'Argentina': 'South America',
    'Poland': 'Europe',
    'Malta': 'Europe',
    'Finland': 'Europe',
    'Estonia': 'Europe',
    'Latvia': 'Europe',
    'Iceland': 'Europe',
    'Czech Republic': 'Europe',
    'Serbia': 'Europe',
    'Andorra': 'Europe',
}

# Assuming 'country_counts_df' is your DataFrame and it has a column named 'Country'
# that matches the keys in the 'continent_mapping'
michelin_df['Continent'] = michelin_df['Country'].map(continent_mapping)

In [11]:
michelin_df.Continent.value_counts()

Europe           4178
Asia             1894
North America     661
South America      61
Name: Continent, dtype: int64

In [12]:
import plotly.express as px

# Assuming michelin_df has a column named 'Continent'
continent_counts = michelin_df['Continent'].value_counts()
# Convert Series to DataFrame for easier manipulation
continent_counts_df = continent_counts.reset_index()
continent_counts_df.columns = ['Continent', 'Listings']

# Since Plotly's built-in continent support can be limited, use country ISO codes and aggregate for continents
# Mapping continents to one or more representative ISO Alpha-3 country codes for visualization purposes
continent_iso_a3 = {
    'Europe': ['FRA', 'DEU', 'ITA', 'ESP', 'GBR', 'BEL', 'CHE', 'NLD', 'PRT', 'DNK', 'SWE', 'AUT', 'HRV', 'NOR', 'GRC', 'SVN', 'LUX', 'HUN', 'POL', 'MLT', 'FIN', 'EST', 'LVA', 'ISL', 'CZE', 'SRB', 'AND', 'IRL'],  # Added European countries
    'Asia': ['CHN', 'JPN', 'IND', 'THA', 'TWN', 'HKG', 'SGP', 'KOR', 'MYS', 'ARE', 'TUR', 'VNM', 'MAC'],  # Added Asian countries
    'North America': ['USA', 'CAN', 'MEX'],  # North America already defined
    'South America': ['BRA', 'ARG', 'COL'],  # South America already defined
}

# Expand the DataFrame to include a row for each ISO code representing a continent
expanded_rows = []
for index, row in continent_counts_df.iterrows():
    for iso in continent_iso_a3.get(row['Continent'], []):
        expanded_rows.append({'ISO_A3': iso, 'Listings': row['Listings'], 'Continent': row['Continent']})
expanded_df = pd.DataFrame(expanded_rows)

# Create a choropleth map using the expanded DataFrame
fig = px.choropleth(expanded_df,
                    locations="ISO_A3",
                    color="Listings",
                    hover_name="Continent",
                    color_continuous_scale=px.colors.sequential.OrRd)

# Update layout for a better view
fig.update_layout(
    title_text='Michelin Guide Listings by Continent',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='natural earth',  # More conventional world map projection
        showcountries=True,  # Outline the countries
        showland=True,  # Show land
        countrywidth=0.5,  # Width of country borders
        landcolor='lightgrey',  # Color of land
        countrycolor='grey',  # Color of country borders
        bgcolor='lightblue',  # Set ocean color to blue
        lakecolor= 'blue'
    ),
    margin={"r":0,"t":50,"l":0,"b":0}  # Adjust margins to fit the layout
)

# Show the figure
fig.show()



In [13]:
michelin_df.Cuisine.value_counts()[:20]

Modern Cuisine              918
Creative                    384
Japanese                    278
Traditional Cuisine         203
Street Food                 168
French                      135
Contemporary                134
Italian                     119
Cantonese                   115
Creative, Modern Cuisine     90
Thai                         80
Classic Cuisine              78
Sushi                        74
Noodles                      72
Modern French                71
Modern British               68
Modern Cuisine, Creative     67
Seafood                      66
Chinese                      57
Farm to table                53
Name: Cuisine, dtype: int64

In [14]:
michelin_df['PrimaryCuisine'] = michelin_df['Cuisine'].str.split(', ').str[0]
michelin_df.PrimaryCuisine.value_counts()[:20]

Modern Cuisine           1197
Creative                  651
Japanese                  362
Contemporary              283
Traditional Cuisine       259
Country cooking           210
French                    188
Italian                   187
Street Food               168
Classic Cuisine           139
Farm to table             126
Modern French             120
Cantonese                 116
Seafood                   113
Mediterranean Cuisine     107
Chinese                   104
Regional Cuisine          103
Thai                       91
Sushi                      74
Noodles                    73
Name: PrimaryCuisine, dtype: int64

---

In [16]:
# Fill NaN values in the 'FacilitiesAndServices' column with an empty string
michelin_df['FacilitiesAndServices'] = michelin_df['FacilitiesAndServices'].fillna('No facility and/or service listed')

# Initialize an empty set to store facilities and services
facilities_and_services = set()

# Iterate through each row
for index, row in michelin_df.iterrows():
    # Check if the value is not NaN
    if not pd.isna(row['FacilitiesAndServices']):
        # Split the values in the 'FacilitiesAndServices' column by comma
        facilities = row['FacilitiesAndServices'].split(',')
        # Add each facility or service to the set
        for facility in facilities:
            facilities_and_services.add(facility.strip())  # Remove leading and trailing whitespace

# Print the set of facilities and services
print(facilities_and_services)

{'Foreign credit cards not accepted', 'Car park', 'Notable sake list', 'Cash only', 'Counter dining', 'Booking essential', 'Bring your own bottle', 'Garden or park', 'Great view', 'Reservations not accepted', 'Brunch', 'Interesting wine list', 'Valet parking', 'Wheelchair access', 'Shoes must be removed', 'Restaurant offering vegetarian menus', 'Cash only - lunch', 'Air conditioning', 'Terrace', 'No facility and/or service listed', 'Credit cards not accepted'}


In [17]:
# Filter rows where the 'FacilitiesAndServices' column includes 'Cash only - lunch'
lunch_rows = michelin_df[michelin_df['FacilitiesAndServices'].str.contains("Cash only - lunch")]

# Display the filtered rows
print(lunch_rows['Award'].value_counts())


Bib Gourmand    21
1 Star          13
2 Stars          2
Name: Award, dtype: int64


---

In [18]:
michelin_df.head(3)

,Name,Address,Location,Price,Cuisine,Longitude,Latitude,PhoneNumber,Url,WebsiteUrl,Award,FacilitiesAndServices,Description,Country,Continent,PrimaryCuisine
0,L'OSIER,"7-5-5 Ginza, Chuo-ku, Tokyo, 104-0061, Japan","Tokyo, Japan",¥¥¥¥,"French, Contemporary",139.761203,35.669667,8.133572e+10,https://guide.michelin.com/en/tokyo-region/tok...,https://losier.shiseido.co.jp/,3 Stars,"Air conditioning,Car park,Interesting wine list",In 2023 L’OSIER celebrated the 50th anniversar...,Japan,Asia,French
1,Azabu Kadowaki,"2-7-2 Azabujuban, Minato-ku, Tokyo, 106-0045, ...","Tokyo, Japan",¥¥¥¥,Japanese,139.733551,35.655993,8.135772e+10,https://guide.michelin.com/en/tokyo-region/tok...,NaN,3 Stars,"Air conditioning,Counter dining",Toshiya Kadowaki’s aim is to serve up “unforge...,Japan,Asia,Japanese
2,Kohaku,"3-4 Kagurazaka, Shinjuku-ku, Tokyo, 162-0825, ...","Tokyo, Japan",¥¥¥¥,Japanese,139.741008,35.700701,8.150314e+11,https://guide.michelin.com/en/tokyo-region/tok...,http://kagurazaka-kohaku.jp/,3 Stars,"Air conditioning,Counter dining,Interesting wi...",Koji Koizumi creates a cuisine all his own as ...,Japan,Asia,Japanese


In [20]:
michelin_df.Country.value_counts().index

Index(['France', 'Japan', 'Italy', 'USA', 'Germany', 'Spain', 'United Kingdom',
       'Belgium', 'Switzerland', 'China Mainland', 'Thailand', 'Netherlands',
       'Taiwan', 'Hong Kong SAR China', 'Singapore', 'South Korea', 'Portugal',
       'Canada', 'Malaysia', 'Brazil', 'Denmark', 'United Arab Emirates',
       'Türkiye', 'Vietnam', 'Ireland', 'Sweden', 'Austria', 'Croatia',
       'Macau SAR China', 'Norway', 'Greece', 'Slovenia', 'Luxembourg',
       'Hungary', 'Argentina', 'Poland', 'Malta', 'Finland', 'Estonia',
       'Czech Republic', 'Latvia', 'Iceland', 'Serbia', 'Andorra'],
      dtype='object')

In [23]:
countries = [
    'France', 'Japan', 'Italy', 'USA', 'Germany', 'Spain', 'United Kingdom',
    'Belgium', 'Switzerland', 'China Mainland', 'Thailand', 'Netherlands',
    'Taiwan', 'Hong Kong SAR China', 'Singapore', 'South Korea', 'Portugal',
    'Canada', 'Malaysia', 'Brazil', 'Denmark', 'United Arab Emirates',
    'Türkiye', 'Vietnam', 'Ireland', 'Sweden', 'Austria', 'Croatia',
    'Macau SAR China', 'Norway', 'Greece', 'Slovenia', 'Luxembourg',
    'Hungary', 'Argentina', 'Poland', 'Malta', 'Finland', 'Estonia',
    'Czech Republic', 'Latvia', 'Iceland', 'Serbia', 'Andorra'
]

# Mapping countries to their currencies for conversion
countries_currencies = [
    'EUR', 'JPY', 'EUR', 'USD', 'EUR', 'EUR', 'GBP', 
    'EUR', 'CHF', 'CNY', 'THB', 'EUR', 
    'TWD', 'HKD', 'SGD', 'KRW', 'EUR', 
    'CAD', 'MYR', 'BRL', 'DKK', 'AED', 
    'TRY', 'VND', 'EUR', 'SEK', 'EUR', 'EUR', 
    'MOP', 'NOK', 'EUR', 'EUR', 'EUR', 
    'HUF', 'ARS', 'PLN', 'EUR', 'EUR', 'EUR', 
    'EUR', 'EUR', 'ISK', 'RSD', 'EUR'
]

# Currency symbols for printing
currency_symbols = {
    'EUR': '€', 'JPY': '¥', 'GBP': '£', 'CHF': 'CHF', 'CNY': '¥',
    'THB': '฿', 'TWD': 'NT$', 'HKD': 'HK$', 'SGD': 'S$', 'KRW': '₩',
    'CAD': 'C$', 'MYR': 'RM', 'BRL': 'R$', 'DKK': 'DKK', 'AED': 'AED',
    'TRY': '₺', 'VND': '₫', 'SEK': 'SEK', 'NOK': 'NOK', 'ISK': 'ISK',
    'USD': '$', 'PLN': 'zł', 'HUF': 'Ft', 'RSD': 'din.', 'MOP': 'MOP$', 'ARS':'ARS$'
}

# Approximate annual average incomes in USD (from the previous list)
annual_average_incomes_usd = [
    41412, 40589, 35354, 65712, 47470, 30371, 42361, 46949, 85582, 10427, 
    7590, 52321, 25848, 49722, 64184, 31977, 23550, 46363, 11589, 8890, 
    60439, 43100, 9277, 2450, 78656, 54108, 50776, 14253, 27972, 81905, 
    21402, 25634, 116681, 15840, 9912, 15848, 33722, 49314, 22984, 22918, 
    19068, 73233, 7320, 40236
]

# Monthly incomes (by dividing annual incomes by 12)
monthly_incomes_usd = [income / 12 for income in annual_average_incomes_usd]

exchange_rates_to_usd = {
    'EUR': 1.080627, # Euro zone
    'JPY': 0.006604, # Japan
    'GBP': 1.259371, # United Kingdom
    'CHF': 1.113458, # Switzerland
    'CNY': 0.138328, # China Mainland
    'THB': 0.027516, # Thailand
    'TWD': 0.031300, # Taiwan
    'HKD': 0.127866, # Hong Kong SAR China
    'SGD': 0.740825, # Singapore
    'KRW': 0.000744, # South Korea
    'CAD': 0.734767, # Canada
    'MYR': 0.211068, # Malaysia
    'BRL': 0.200020, # Brazil
    'DKK': 0.144886, # Denmark
    'AED': 0.272294, # United Arab Emirates
    'TRY': 0.031218, # Türkiye
    'VND': 0.000043, # Vietnam
    'SEK': 0.094603, # Sweden
    'NOK': 0.092992, # Norway
    'ISK': 0.007255, # Iceland
    'USD': 1, # USA and other USD users
    'PLN': 0.249916, # Poland
    'HUF': 0.002717, # Hungary
    'RSD': 0.009219, # Serbia
    'MOP': 0.124134, # Macau SAR China
    'ARS': 0.001170, # Argentine Peso
}

# Converting monthly incomes from USD to local currency using exchange rates
rounded_monthly_incomes_local = [
    round(income_usd / exchange_rates_to_usd[currency], 2)
    for income_usd, currency in zip(monthly_incomes_usd, countries_currencies)
]

# Printing country name and respective local currency monthly average income
for country, income, currency in zip(countries, rounded_monthly_incomes_local, countries_currencies):
    print(f"{country}: {currency_symbols[currency]}{income}")


France: €3193.52
Japan: ¥512176.96
Italy: €2726.35
USA: $5476.0
Germany: €3660.68
Spain: €2342.08
United Kingdom: £2803.05
Belgium: €3620.51
Switzerland: CHF6405.12
China Mainland: ¥6281.57
Thailand: ฿22986.63
Netherlands: €4034.77
Taiwan: NT$68817.89
Hong Kong SAR China: HK$32405.02
Singapore: S$7219.88
South Korea: ₩3581653.23
Portugal: €1816.08
Canada: C$5258.24
Malaysia: RM4575.54
Brazil: R$3703.8
Denmark: DKK34762.39
United Arab Emirates: AED13190.4
Türkiye: ₺24764.03
Vietnam: ₫4748062.02
Ireland: €6065.61
Sweden: SEK47662.34
Austria: €3915.63
Croatia: €1099.13
Macau SAR China: MOP$18778.09
Norway: NOK73397.89
Greece: €1650.43
Slovenia: €1976.78
Luxembourg: €8997.94
Hungary: Ft485829.96
Argentina: ARS$705982.91
Poland: zł5284.44
Malta: €2600.5
Finland: €3802.88
Estonia: €1772.43
Czech Republic: €1767.34
Latvia: €1470.44
Iceland: ISK841178.5
Serbia: din.66167.7
Andorra: €3102.83


In [8]:
michelin_df[michelin_df["Price"].isna()]

,Name,Address,Location,Price,Cuisine,Longitude,Latitude,PhoneNumber,Url,WebsiteUrl,Award,FacilitiesAndServices,Description,Country
4634,Meiers come inn,"Kaffeestrasse 6c, Bülach, 8180, Switzerland","Bülach, Switzerland",NaN,Thai,8.536827,47.513068,4.144860e+10,https://guide.michelin.com/en/zurich-region/bl...,https://www.come-inn.ch/,Bib Gourmand,"Air conditioning,Car park,Restaurant offering ...","Slightly off the beaten track, this restaurant...",Switzerland
6761,Nit,"Rua Oscar Freire 145, São Paulo, 01409-003, Br...","São Paulo, Brazil",NaN,Spanish,-46.663807,-23.567758,5.511354e+11,https://guide.michelin.com/en/sao-paulo-region...,https://www.restaurantetanit.com.br/cardapios-nit,Bib Gourmand,Air conditioning,"The most informal and fun of Catalan chef, Osc...",Brazil


In [9]:
na_price_indices = michelin_df[michelin_df['Price'].isna()].index.tolist()
na_price_indices

[4634, 6761]

In [10]:
# Fill these indices with a 'Price' of 100
michelin_df.loc[4634, 'Price'] = "$$"
michelin_df.loc[6761, 'Price'] = "$$$"

In [18]:
michelin_df["Price"].value_counts()[:20]

€€€€    1387
€€      1339
€€€      798
$$       572
$$$$     336
¥¥¥      330
€        307
¥¥       268
$        267
¥        201
¥¥¥¥     176
$$$      145
฿฿       115
££££     114
££        98
฿         81
£££       59
₩         49
₩₩₩₩      26
₫         25
Name: Price, dtype: int64

---